<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [179]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split , cross_val_score ,cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [180]:
house = pd.read_csv('./housing.csv')

In [181]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [182]:
#C (all) is non residential
house.MSZoning.value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [183]:
#drop all non-residential entries
drop_list = list(house[house['MSZoning']=="C (all)"].index)
house.drop(index = drop_list,inplace = True)

In [184]:
#create new feature age_sold before removing the columns
house['age_sold'] = house['YrSold']-house['YearRemodAdd']

In [185]:
#drop columns that is not a characteristic of house
not_house_char = ['Id','MoSold','YearBuilt','YearRemodAdd','SaleType','GarageYrBlt']
house.drop(columns =(not_house_char) , inplace = True)


In [186]:
#split 'fixed' and 'not fixed' variables.

fixed_var = ['MSSubClass','MSZoning','LotFrontage','LotArea','Street','Alley','LotShape',
             'LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1',
             'Condition2','BldgType','HouseStyle','RoofStyle','MasVnrArea','Foundation',
             'BsmtExposure','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','BsmtFullBath',
             'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd',
             'Fireplaces','GarageType','GarageCars','GarageArea','PavedDrive','WoodDeckSF',
             'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',
             'MiscFeature','MiscVal','SaleCondition','SalePrice','YrSold','age_sold']
not_fixed = [i for i in house.columns if i not in fixed_var]

#columns that will be used to train Q1 model.
house_fixed = house[fixed_var]

#set aside columns 
house_renov = house[not_fixed]


In [187]:
house_fixed.isnull().sum().sort_values(ascending= False).head()

MiscFeature     1398
Alley           1361
LotFrontage      259
GarageType        79
BsmtExposure      38
dtype: int64

In [188]:
#Fixed the easy null values first. The null values have meanings. 
house_fixed.loc[house_fixed['Alley'].isnull(),"Alley"] = "N"
house_fixed.loc[house_fixed['GarageType'].isnull(),"GarageType"] = "N"
house_fixed.loc[house_fixed['BsmtExposure'].isna(),"BsmtExposure"] = "N"

c:\users\chenz\anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [195]:
house_fixed[house_fixed['MiscFeature'].isnull()]["MiscVal"].value_counts()

0    1398
Name: MiscVal, dtype: int64

In [161]:
#Decided to drop Misc Feature as misc value will be able to represent it.
house_fixed.drop(columns = 'MiscFeature',inplace = True)

c:\users\chenz\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [162]:
#decided to impute null value with mean
frontage_mean = np.mean(house_fixed['LotFrontage'].dropna())
#impute na with mean value
house_fixed.loc[house_fixed['LotFrontage'].isna(),"LotFrontage"] = frontage_mean


c:\users\chenz\anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [163]:
#check why MasVnrArea has null. Retrieve columns from main dataframe
house[house['MasVnrArea'].isnull()]['MasVnrType']

234     NaN
529     NaN
650     NaN
936     NaN
973     NaN
977     NaN
1243    NaN
1278    NaN
Name: MasVnrType, dtype: object

In [164]:
#MasVnrArea is Null value, because MasVnrType is None, meaning no Masonry Veneer
house_fixed['MasVnrArea'] = house_fixed['MasVnrArea'].replace(np.nan,0)

c:\users\chenz\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [165]:
house_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Data columns (total 47 columns):
MSSubClass       1450 non-null int64
MSZoning         1450 non-null object
LotFrontage      1450 non-null float64
LotArea          1450 non-null int64
Street           1450 non-null object
Alley            1450 non-null object
LotShape         1450 non-null object
LandContour      1450 non-null object
Utilities        1450 non-null object
LotConfig        1450 non-null object
LandSlope        1450 non-null object
Neighborhood     1450 non-null object
Condition1       1450 non-null object
Condition2       1450 non-null object
BldgType         1450 non-null object
HouseStyle       1450 non-null object
RoofStyle        1450 non-null object
MasVnrArea       1450 non-null float64
Foundation       1450 non-null object
BsmtExposure     1450 non-null object
TotalBsmtSF      1450 non-null int64
1stFlrSF         1450 non-null int64
2ndFlrSF         1450 non-null int64
GrLivArea        1450 

In [166]:
#found mssubclass wrong type
house_fixed['MSSubClass'] = house_fixed['MSSubClass'].astype(str)

c:\users\chenz\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [173]:
#get a list of all object column and apply get_dummies
categorical = list(house_fixed.select_dtypes(include=['object']).columns)
df_dummies = pd.get_dummies(house_fixed[categorical], drop_first=True)
len(df_dummies.columns)

110

In [174]:
house_fixed= house_fixed.drop(columns = categorical)

In [175]:
house_fixed = pd.concat([house_fixed, df_dummies], axis=1)

In [176]:
len(house_fixed.columns)

137

In [177]:
house_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1459
Columns: 137 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(2), int64(25), uint8(110)
memory usage: 472.9 KB


In [145]:
#split data into 2010(test data) and before 2010(training data)
house_fixed_2010 = house_fixed[house_fixed['YrSold'] == 2010]
y_test = house_fixed_2010[['SalePrice']]
X_test = house_fixed_2010.drop(columns = 'SalePrice')

house_fixed_training = house_fixed[house_fixed['YrSold'] < 2010]
y_train = house_fixed_training[['SalePrice']]
X_train = house_fixed_training.drop(columns = 'SalePrice')

In [146]:
#perform automatic feature selection method "SelectKBest"
from sklearn.feature_selection import SelectKBest,f_regression
cols = list(X_train.columns)
# Build the selector — we'll build one with each score type.
skb_f = SelectKBest(f_regression)

# Train the selector on the data.
skb_f.fit(X_train, y_train)

# Examine the results.
kbest = pd.DataFrame([cols, list(skb_f.scores_),list(skb_f.pvalues_)], 
                     index=['feature','f_regression','p_values']).T.sort_values('f_regression', ascending=False)
kbest

c:\users\chenz\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,feature,f_regression,p_values
6,GrLivArea,1284,3.75489e-195
15,GarageCars,894.937,1.96186e-149
16,GarageArea,844.791,5.96242e-143
3,TotalBsmtSF,731.737,9.40283e-128
4,1stFlrSF,711.931,5.2963e-125
9,FullBath,590.937,1.41251e-107
13,TotRmsAbvGrd,495.105,5.99454e-93
25,age_sold,443.142,1.11217e-84
115,Foundation_PConc,387.207,1.71151e-75
14,Fireplaces,358.346,1.24868e-70


In [147]:
#Select range of features by 10 by f_regression score
top_list=[]
for i in range(10,110,10):
    top =[]
    top = list(kbest.feature.head(i))
    top_list.append(top)
    
len(top_list)

10

In [149]:
#Apply linear regression model on the features selected by using kbest_f_regression
#Run through different number of features. 
for ls in top_list:
    print("===",len(ls), "Features===")
#     Xs = ss.fit_transform(X_train[ls])
#     Xs_test = ss.transform(X_test[ls])
    X_ = X_train[ls]
    X_test_ = X_test[ls]
    model = LinearRegression()
    # Perform cross-validation
    scores = cross_val_score(model,X_, y_train, cv=10)
    print("Cross-validated scores:", scores)
    print("Mean CV R2:", np.mean(scores))
    print('Std CV R2:', np.std(scores))

    model.fit(X_,y_train)

    print("Test Score:" ,model.score(X_test_,y_test))
    print("")

=== 10 Features===
Cross-validated scores: [0.78988838 0.78994704 0.81220428 0.68130621 0.74974434 0.74835443
 0.77760399 0.72808072 0.36934983 0.77248042]
Mean CV R2: 0.7218959634234825
Std CV R2: 0.12268733280910674
Test Score: 0.7705994927517439

=== 20 Features===
Cross-validated scores: [0.80783457 0.80380277 0.80472646 0.71609372 0.78270116 0.83049627
 0.79562347 0.72903198 0.44294451 0.81409912]
Mean CV R2: 0.7527354022735703
Std CV R2: 0.1089822560565189
Test Score: 0.7845149111087549

=== 30 Features===
Cross-validated scores: [0.83752363 0.8031554  0.81680767 0.72427892 0.79976168 0.82668301
 0.81342386 0.74475944 0.47099614 0.83160656]
Mean CV R2: 0.7668996293491314
Std CV R2: 0.10472062431521317
Test Score: 0.8117879112435716

=== 40 Features===
Cross-validated scores: [0.84228522 0.82736631 0.84462345 0.74473075 0.79601461 0.82932608
 0.81845456 0.78204409 0.50217729 0.84766653]
Mean CV R2: 0.783468889383527
Std CV R2: 0.09867090431658272
Test Score: 0.8374052472692151

==

Achieved the best score when top 90 features use being applied. 

Linear Regression ( No Scaling ) 

=== 90 Features===

Cross-validated scores: 
-[0.86880826 0.85507781 0.86194763 0.73846059 0.81454982 0.66356699 0.84258537 0.80280333 0.51671148 0.84275877]

-Mean CV R2: 0.7807270046401211

-Std CV R2: 0.10696074043906398

-Test Score: 0.8618559858047153


In [ ]:
#Apply linear regression model on the features selected by using kbest_f_regression
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

#Run through different set of features. 
for ls in top_list:
    print("===",len(ls), "Features===")
#     Xs = ss.fit_transform(X_train[ls])
#     Xs_test = ss.transform(X_test[ls])
    X_ = X_train[ls]
    X_test_ = X_test[ls]
    model = LinearRegression()
    # Perform cross-validation
    scores = cross_val_score(model,X_, y_train, cv=10)
    print("Cross-validated scores:", scores)
    print("Mean CV R2:", np.mean(scores))
    print('Std CV R2:', np.std(scores))

    model.fit(X_,y_train)

    print("Test Score:" ,model.score(X_test_,y_test))
    print("")

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
Xs = ss.fit_transform(X)
Xs_test = ss.transform(X_test)
optimal_lasso = LassoCV(n_alphas=500, cv=10)
optimal_lasso.fit(Xs, y)
print(optimal_lasso.alpha_)

In [ ]:
lasso = Lasso(alpha=optimal_lasso.alpha_)
lasso_scores = cross_val_score(lasso, Xs, y, cv=10)

print(lasso_scores)
print(np.mean(lasso_scores))

lasso.fit(Xs,y)
lasso.score(Xs_test,y_test)

lasso_coefs = pd.DataFrame({'variable':X.columns,
                            'coef':lasso.coef_,
                            'abs_coef':np.abs(lasso.coef_)})

lasso_coefs.sort_values('coef', inplace=True, ascending=False)

lasso_coefs[lasso_coefs['coef']!=0]

In [ ]:
ridge_alphas = np.logspace(-2, 7, 50)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(Xs, y)

print(optimal_ridge.alpha_)

In [ ]:
ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, Xs, y, cv=10)

print(ridge_scores)
print('Ridge Training Mean:',np.mean(ridge_scores))

ridge.fit(Xs, y)
print('Ridge Predict Score:', ridge.score(Xs_test,y_test))

# ridge_coef = ridge.coef_[0][:]
# ridge_coefs = pd.DataFrame({'variable':X.columns,
#                             'coef':ridge_coef,
#                             'abs_coef':np.abs(ridge_coef)})

# ridge_coefs.sort_values('abs_coef', inplace=True, ascending=False)

# ridge_coefs[ridge_coefs['coef']!=0]

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)

optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, 
                            n_alphas=30, 
                            cv=10, 
                            max_iter=10000)
optimal_enet.fit(Xs, y_train)

print(optimal_enet.alpha_)
print(optimal_enet.l1_ratio_)

In [ ]:
enet = ElasticNet(alpha=optimal_enet.alpha_, 
                  l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, Xs, y_train, cv=10)

print(enet_scores)
print(np.mean(enet_scores))

enet.fit(Xs, y_train)
enet_coef = enet.coef_
enet_coefs_df = pd.DataFrame({'variable':X.columns,
                            'coef':enet_coef,
                            'abs_coef':np.abs(enet_coef)})

enet_coefs_df.sort_values('abs_coef', inplace=True, ascending=False)
enet_coefs_df

In [ ]:
print(np.mean(enet_scores))
print(np.mean(ridge_scores))
print(np.mean(lasso_scores))

In [ ]:
comparison = pd.concat([enet_coefs_df,kbest],axis=1)
comparison.sort_values(by='coef',ascending = False)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [ ]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# A: